In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
from collections import Counter
from itertools import chain
import pickle

import pandas as pd
from ipywidgets.widgets import HTML, Output, HBox, Button, Text
from IPython.display import display, clear_output
import json

from mimic_note_clean import parse_radiology_report_text, parse_echo_report_text, parse_nurs_prog_notes_text, \
    parse_phys_prog_notes_text, parse_phys_intens_notes_text, parse_phys_attend_notes_text, parse_phys_res_attnd_adm_notes_text, \
    parse_phys_res_attn_micu_notes_text, parse_phys_res_attn_prog_notes_text, parse_phys_surgical_adm_notes_text, \
    parse_respiratory_care_shift_note
from section_parser import section_discharge_texts, dis_sum_brief_hos_course_headers

In [1057]:
stroke_notes = pd.read_csv('../../data/43411_notes.csv')

In [4]:
stroke_notes.loc[:, ['category', 'description', 'text']]

,category,description,text
0,Nursing,Nursing Progress Note,"CVA (Stroke, Cerebral infarction), Ischemic\n ..."
1,Nursing,Nursing Progress Note,"CVA (Stroke, Cerebral infarction), Ischemic\n ..."
2,Nursing,Nursing Progress Note,"CVA (Stroke, Cerebral infarction), Ischemic\n ..."
3,Physician,Physician Surgical Admission Note,Chief Complaint: Stroke\n HPI:\n HPI: 84 ...
4,Nursing,Nursing Progress Note,"CVA (Stroke, Cerebral infarction), Ischemic\n ..."
...,...,...,...
8208,Nursing,Nursing Progress Note,"TITLE:\n CVA (Stroke, Cerebral infarction), ..."
8209,Nursing,Nursing Progress Note,".H/O airway, Inability to Protect (Risk for As..."
8210,Respiratory,Respiratory Care Shift Note,Demographics\n Day of intubation:\n Day of...
8211,Physician,Intensivist Note,SICU\n HPI:\n 76 yo F h/o afib on Coumadin...


In [5]:
stroke_notes['cat_desc'] = stroke_notes.category.str.cat(stroke_notes.description, sep=':')

In [5]:
stroke_notes.cat_desc.unique()

NameError: name 'stroke_notes' is not defined

In [7]:
texts = stroke_notes[stroke_notes.cat_desc == 'Discharge summary:Report'].text.str.lower().reset_index(drop=True)

In [35]:
all_discharge_notes = pd.read_csv('/Users/tom/phd/tdy_ehr/mimic_tidy/data/notes.csv')

In [36]:
admissions = pd.read_csv('/Users/tom/phd/tdy_ehr/mimic_tidy/data/admissions.csv')

In [37]:
patients = pd.read_csv('/Users/tom/phd/tdy_ehr/mimic_tidy/data/patients.csv')
patients.dob = patients.dob.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [38]:
all_discharge_notes = all_discharge_notes.merge(patients.loc[:, ['subject_id', 'dob']], 'left', 'subject_id')

In [39]:
all_discharge_notes['chartdate'] = all_discharge_notes.chartdate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

Index(['Unnamed: 0', 'subject_id', 'hadm_id', 'chartdate', 'category',
       'description', 'text', 'dob'],
      dtype='object')

In [45]:
all_discharge_notes.drop('Unnamed: 0', inplace=True, axis=1)

In [46]:
all_discharge_notes['subject_age'] = all_discharge_notes.apply(lambda r: r.chartdate.year - r.dob.year, axis=1)

In [47]:
pat = [
    '(brief hospital course:)',
    '(hospital course:)',
    '(concise summary of hospital course by issue/system:)',
    '(summary of hospital course by systems:)',
    '(hospital course by systems including pertinent laboratory\ndata:)',
    '(details of hospital course:)',
]
contains_brief_hosp_course_sec = all_discharge_notes.text.str.lower().str.contains('|'.join(pat))
brief_hos_course_notes = all_discharge_notes[contains_brief_hosp_course_sec]

/Users/tom/anaconda3/envs/cattrainer/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [48]:
comp_pat = re.compile('|'.join(pat), re.IGNORECASE)

In [49]:
matches = all_discharge_notes.text.apply(lambda t: re.search(comp_pat, t))

In [50]:
dis_notes = all_discharge_notes.loc[~pd.isna(matches)]

In [51]:
matches = matches.loc[~pd.isna(matches)]

In [ ]:
matches.shape

In [15]:
dis_notes['match_start'] = matches

/Users/tom/anaconda3/envs/cattrainer/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
dis_notes['hos_course_text'] = dis_notes.apply(lambda r: r.text[r.match_start.start():], axis=1)

/Users/tom/anaconda3/envs/cattrainer/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
end_delim_match = [re.search('(\n\n(?!#|\*|\d|\s|\[|\().*:)', s, re.IGNORECASE) for s in samples]

In [81]:
dis_notes.sample(100).to_csv('dis_notes_sample.csv')

In [ ]:
dis_notes['hos_course_text_subsection'] = dis_notes.apply(lambda r: r.hos_course_text[0:r.next_sec_matches.start()], axis=1)

In [ ]:
dis_notes.shape

In [469]:
full_sec = random_100_sample.iloc[idx].hos_course_text
sec = random_100_sample.iloc[idx].hos_course_text_subsection
HTML(value=f'<div style="width=100%">' +\
     f'<div style="display:inline-block;width:50%; white-space: pre-wrap; line-height: 1.2">{full_sec}</div>' +\
     f'<div style="display:inline-block;width:50%; white-space: pre-wrap; line-height: 1.2">{sec}</div>' +\
     '</div>')

HTML(value='<div style="width=100%"><div style="display:inline-block;width:50%; white-space: pre-wrap; line-he…

In [ ]:
# sorted(Counter([e.group(0).lower() if e is not None else '' for e in end_delim_match]).items(), key=lambda k_v: k_v[1], reverse=True) 
# set([s.groups(0)[0] for i, s in enumerate(samples_short) if s is not None ])

In [ ]:
dis_notes.drop('Unnamed: 0', axis=1, inplace=True)

In [708]:
dis_notes.reset_index(drop=True).to_csv('dis_notes_hosp_course_extracted.csv')

In [193]:
# Retrieve DataFrame
dis_notes = pd.read_csv('dis_notes_hosp_course_extracted.csv')

In [194]:
dis_notes.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
# Remove:
# - re-admissions, cases that have the same admission ID but with multiple discharge summary reports
# - where there are addendums to exisiting reports

In [117]:
pickle.dump(out_texts, open('dis_course_secs.pickle', 'wb'))

In [113]:
out_texts = dis_notes.text.apply(section_discharge_texts)

In [134]:
brief_hos_course_secs = []
other_secs = []
dis_course_headers = [h.lower().strip().replace(':', '') for h in dis_sum_brief_hos_course_headers]
for row in out_texts:
    brief_hos_course = None
    brief_hos_course_idx = None
    for i, (sec, sec_name) in enumerate(zip(row[0], row[1])):
        if sec_name in dis_course_headers:
            brief_hos_course = sec
            brief_hos_course_idx = i
    if brief_hos_course_idx is not None:
        row_secs = row[0][0:brief_hos_course_idx] + row[0][brief_hos_course_idx+1:]
        row_sec_headers = row[1][0:brief_hos_course_idx] + row[1][brief_hos_course_idx+1:]
    brief_hos_course_secs.append(brief_hos_course)
    other_secs.append([row_secs, row_sec_headers])

In [139]:
other_sec_header_counts = Counter(list(chain.from_iterable([o[1] for o in other_secs])))

In [170]:
sec_headers_df = pd.Series(other_sec_header_counts).sort_values(ascending=False).reset_index()
print(sec_headers_df.shape)
hdrs = sec_headers_df['index'].tolist()

(83, 2)


In [ ]:
print('\n'.join([f'{h}:{c}' for h,c in zip(sec_headers_df['index'].tolist(), sec_headers_df[0].tolist())]))

In [178]:
headers_to_use = [
    "preamble",
    "history of present illness",
    "past medical history",
    "discharge medications",
    "physical exam",
    "medications on admission",
    "discharge diagnosis",
    "discharge disposition",
    "impression",
    "physical examination",
    "hpi",
    "problem list",
]

In [209]:
list(zip(out_texts[0])

(['preamble',
  'history of present illness',
  'past medical history',
  'physical exam',
  'brief hospital course',
  'medications on admission',
  'discharge medications',
  'discharge disposition',
  'discharge diagnosis',
  'discharge condition',
  'discharge instructions'],)

In [248]:
filtered_out_texts = []
for secs, sec_headers, _ in out_texts:
    filtered_secs, filtered_headers = [], []
    for sec, sec_h in zip(secs, sec_headers):
        if sec_h in headers_to_use:
            filtered_secs.append(sec)
            filtered_headers.append(sec_h)
    filtered_out_texts.append([filtered_secs, filtered_headers])

In [257]:
dis_notes.columns

Index(['subject_id', 'hadm_id', 'chartdate', 'category', 'description', 'text',
       'dob', 'subject_age', 'match_start', 'hos_course_text',
       'next_sec_matches', 'hos_course_text_subsection'],
      dtype='object')

In [ ]:
dis_notes.loc[:, ['hos_course_text_subsection', 'hadm_id']]

In [ ]:
filtered_out_texts

In [264]:
len(filtered_out_texts)

51267

In [267]:
dis_notes['other_dis_notes_secs'] = filtered_out_texts

In [269]:
dis_notes.to_csv('dis_notes_hosp_course_extracted.csv')

In [9]:
all_notes = pd.read_csv('/Users/tom/phd/tdy_ehr/mimic_tidy/data/all_notes.csv')

/Users/tom/anaconda3/envs/cattrainer/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
print(all_notes[all_notes.hadm_id == 117390].iloc[0].text)

In [10]:
all_notes = all_notes[all_notes.hadm_id.isin(single_adm_ids)]

In [82]:
# remove discharge summaries
all_notes = all_notes[all_notes.category != 'Discharge summary']

In [388]:
# remove ECG reports
# ECG text is pretty short and not that useful...
all_notes = all_notes[all_notes.category != 'ECG']

In [11]:
# Remove:
# - single report admissions. These are only discharge summaries so cannot be 'summarised' from other notes.
single_note_admissions = (all_notes.hadm_id.value_counts() == 1).reset_index()
single_note_admissions = single_note_admissions[single_note_admissions.hadm_id]['index'].reset_index(drop=True)
all_notes = all_notes[~all_notes.hadm_id.isin(single_note_admissions)]

In [ ]:
dis_notes[dis_notes.hadm_id == 175441]

In [ ]:
print(dis_notes[dis_notes.hadm_id == 118785].hos_course_text_subsection.iloc[])

In [120]:
len([(k,v) for k,v in Counter(dis_notes.hadm_id).items() if v > 1])

1579

In [124]:
all_notes.columns

Index(['subject_id', 'hadm_id', 'chartdate', 'charttime', 'text', 'category',
       'description'],
      dtype='object')

In [35]:
all_notes['cat:desc'] = all_notes.category.str.cat(all_notes.description, sep=':')

In [39]:
all_notes['cat:desc'].value_counts().head(10)

Nursing/other:Report                           639820
Nursing:Nursing Progress Note                  186587
Radiology:CHEST (PORTABLE AP)                  148309
ECG:Report                                     131213
Physician :Physician Resident Progress Note     61453
Discharge summary:Report                        51449
Echo:Report                                     32262
Respiratory :Respiratory Care Shift Note        30575
Nursing:Nursing Transfer Note                   29642
Physician :Intensivist Note                     25378
Name: cat:desc, dtype: int64

In [149]:
chest_ap_notes = all_notes[all_notes['cat:desc'] == 'Radiology:CHEST (PORTABLE AP)'].text
idx = 0

In [60]:
# Final 'clean' doc to be fed into a model.
all_notes['clean_text'] = None

In [256]:
all_notes.loc[:, 'charttime'] = all_notes.charttime.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S') if type(x) == str else x)

/Users/tom/anaconda3/envs/cattrainer/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [258]:
all_notes[:, 'chartdate'] = all_notes.chartdate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

0         2126-12-01 17:22:00
1         2165-12-08 17:12:00
2         2165-12-08 17:12:00
3         2165-12-08 17:12:00
4         2150-11-30 10:58:00
                  ...        
2083175   2150-11-29 20:18:00
2083176   2190-12-06 10:52:00
2083177   2190-12-06 10:52:00
2083178   2126-12-01 17:22:00
2083179   2203-11-01 07:54:00
Name: charttime, Length: 1572322, dtype: datetime64[ns]

In [763]:
parsed_reports = parse_radiology_report_text(all_notes.loc[all_notes.category == 'Radiology'])

In [764]:
# only use the 'impression' sections of radiology reports
all_notes.loc[all_notes.category == 'Radiology', 'clean_text'] = [rep[0] for rep in parsed_reports]

/Users/tom/anaconda3/envs/cattrainer/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [765]:
radio_notes = all_notes.loc[all_notes.category == 'Radiology', 'clean_text']
no_imp_radio_notes = all_notes.loc[radio_notes[pd.isna(radio_notes)].index]

In [332]:
all_notes.to_csv('all_notes_partially_parsed.csv')

In [373]:
echo_impressions = parse_echo_report_text(all_notes[all_notes.category == 'Echo'])
# only use the 'impression' sections of radiology reports
all_notes.loc[all_notes.category == 'Echo', 'clean_text'] = [rep[0] for rep in echo_impressions]

In [903]:
nurs_prog_notes = all_notes.loc[(all_notes.category == 'Nursing') & (all_notes.description == 'Nursing Progress Note')]

In [904]:
nurs_prog_reports_assessments = parse_nurs_prog_notes_text(nurs_prog_notes)

/Users/tom/phd/summariser/clinical_sum/mimic_summarisation_clean/mimic_radiology_clean.py:18: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  else:


In [907]:
assessment_reports = [rep[0].strip() for rep in nurs_prog_reports_assessments]

In [912]:
row_idxr = (all_notes.category == 'Nursing') & (all_notes.description == 'Nursing Progress Note')
all_notes.loc[row_idxr, 'clean_text'] = assessment_reports

In [837]:
nurs_not_prog_notes = all_notes[(all_notes.category == 'Nursing') & (all_notes.description != 'Nursing Progress Note')]

In [943]:
nurs_transfer_notes_assessments = parse_nurs_prog_notes_text(nurs_not_prog_notes[nurs_not_prog_notes.description == 'Nursing Transfer Note'])

/Users/tom/phd/summariser/clinical_sum/mimic_summarisation_clean/mimic_radiology_clean.py:18: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  else:


In [948]:
assessment_reports = [rep[0].strip() for rep in nurs_transfer_notes_assessments]

In [949]:
row_idxr = (all_notes.category == 'Nursing') & (all_notes.description == 'Nursing Transfer Note')
all_notes.loc[row_idxr, 'clean_text'] = assessment_reports

/Users/tom/anaconda3/envs/cattrainer/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [268]:
all_notes[pd.isna(all_notes.clean_text)].category.value_counts()

Radiology           102964
Nursing              35043
Physician            10284
Nutrition             9233
General               8043
Rehab Services        5343
Social Work           2541
Case Management        942
Pharmacy               102
Consult                 98
Respiratory             42
Echo                    12
Name: category, dtype: int64

In [ ]:
row_idxr = (all_notes.category == 'Physician ') & (all_notes.description == 'Physician Resident Progress Note')
all_notes.loc[row_idxr, 'clean_text'] = [sec[0] for sec in parse_phys_prog_notes_text(all_notes.loc[row_idxr, ['text']])]

In [90]:
all_notes.drop('Unnamed: 0', axis=1, inplace=True)

In [91]:
all_notes.head(5)

,subject_id,hadm_id,chartdate,charttime,text,category,description,cat:desc,clean_text
0,27121,167456.0,2126-12-01 00:00:00,2126-12-01 17:22:00,Pt. is a 60 year old with a history of large B...,Nursing,Nursing Progress Note,Nursing:Nursing Progress Note,Pt had been hemodynamically stable but around ...
1,6850,167393.0,2165-12-08 00:00:00,2165-12-08 17:12:00,Ms. [**Known lastname 7005**] is a 36 y/o woma...,Nursing,Nursing Progress Note,Nursing:Nursing Progress Note,Na 159 this morning\n \n\n BUN 33 (42) and...
2,6850,167393.0,2165-12-08 00:00:00,2165-12-08 17:12:00,Ms. [**Known lastname 7005**] is a 36 y/o woma...,Nursing,Nursing Progress Note,Nursing:Nursing Progress Note,Na 159 this morning\n \n\n BUN 33 (42) and...
3,6850,167393.0,2165-12-08 00:00:00,2165-12-08 17:12:00,Ms. [**Known lastname 7005**] is a 36 y/o woma...,Nursing,Nursing Progress Note,Nursing:Nursing Progress Note,Na 159 this morning\n \n\n BUN 33 (42) and...
4,27427,175448.0,2150-11-30 00:00:00,2150-11-30 10:58:00,Planned Discharge Date: [**2150-11-30**]\n I...,Case Management,Case Management Continued Stay Review,Case Management :Case Management Continued Sta...,NaN


In [267]:
all_notes.to_csv('all_notes_partially_parsed.csv', index=False)

In [ ]:
all_notes = pd.read_csv('all_notes_partially_parsed.csv')

In [ ]:
[print(n) for n in dis_notes.sample(1).text]

In [ ]:
print(all_notes[all_notes.hadm_id == 117390.0].iloc[0].text)

In [4]:
all_notes[pd.isna(all_notes.clean_text)].category.value_counts()

Radiology           102964
Nursing              35043
Physician            12638
Nutrition             9233
General               8043
Rehab Services        5343
Respiratory           3208
Social Work           2541
Case Management        942
Pharmacy               102
Consult                 98
Echo                    12
Name: category, dtype: int64

Index(['subject_id', 'hadm_id', 'chartdate', 'charttime', 'text', 'category',
       'description', 'cat:desc', 'clean_text'],
      dtype='object')

In [7]:
all_notes[~pd.isna(all_notes.clean_text)].category.value_counts()

Nursing/other    639820
Radiology        253549
Nursing          182093
Physician        125233
Echo              32250
Respiratory       27997
Name: category, dtype: int64

In [123]:
row_idxr = (all_notes.category == 'Physician ') & (all_notes.description == 'Intensivist Note')

In [124]:
all_notes.loc[row_idxr, ['clean_text']] = [sec[0] for sec in parse_phys_intens_notes_text(all_notes.loc[row_idxr, ['text']])]

In [84]:
row_idxr = (all_notes.category == 'Physician ') & (all_notes.description == 'Physician Attending Progress Note')

In [86]:
all_notes.loc[row_idxr, ['clean_text']] = [sec[0] for sec in parse_phys_attend_notes_text(all_notes.loc[row_idxr, ['text']])]

/Users/tom/phd/summariser/clinical_sum/mimic_summarisation_clean/mimic_radiology_clean.py:21: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  else:


In [116]:
row_idxr = (all_notes.category == 'Physician ') & (all_notes.description == 'Physician Resident Admission Note')

In [117]:
all_notes.loc[row_idxr, ['clean_text']] = [sec[0] for sec in parse_phys_res_attnd_adm_notes_text(all_notes.loc[row_idxr, ['text']])]

In [118]:
row_idxr = (all_notes.category == 'Physician ') & (all_notes.description == 'ICU Note - CVI')

In [119]:
all_notes.loc[row_idxr, ['clean_text']] = [sec[0] for sec in parse_phys_intens_notes_text(all_notes.loc[row_idxr, ['text']])]

/Users/tom/phd/summariser/clinical_sum/mimic_summarisation_clean/mimic_radiology_clean.py:21: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  idxs = np.argwhere(np.array(section_names) == section_name).flatten()


In [120]:
row_idxr = (all_notes.category == 'Physician ') & (all_notes.description == 'Physician Attending Admission Note - MICU')

In [121]:
all_notes.loc[row_idxr, 'clean_text'] = [sec[0] for sec in parse_phys_res_attn_micu_notes_text(all_notes.loc[row_idxr, ['text']])]

In [126]:
row_idxr = (all_notes.category == 'Physician ') & (all_notes.description == 'Physician Resident/Attending Progress Note - MICU')

In [134]:
all_notes.loc[row_idxr, 'clean_text'] = [sec[0] for sec in parse_phys_res_attn_prog_notes_text(all_notes.loc[row_idxr, ['text']])]

In [158]:
row_idxr = (all_notes.category == 'Physician ') & ((all_notes.description == 'Physician Surgical Admission Note') |  
                                                   (all_notes.description == 'Physician Fellow / Attending Progress Note - MICU'))

In [163]:
all_notes.loc[row_idxr, 'clean_text'] = [sec[0] for sec in parse_phys_surgical_adm_notes_text(all_notes.loc[row_idxr, ['text']])]

In [151]:
row_idxr = (all_notes.category == 'Physician ') &  (all_notes.description == 'Physician Fellow / Attending Progress Note - MICU')

In [155]:
# can use the same func
all_notes.loc[row_idxr, 'clean_text'] = [sec[0] for sec in parse_phys_surgical_adm_notes_text(all_notes.loc[row_idxr, ['text']])]

In [219]:
row_idxr = (all_notes.category == 'Respiratory ') & ((all_notes.description == 'Respiratory Care Shift Note') | 
                                                     (all_notes.description == 'Generic Note'))

In [220]:
all_notes.loc[row_idxr, 'clean_text'] = [sec[0] for sec in parse_respiratory_care_shift_note(all_notes.loc[row_idxr, ['text']])]

/Users/tom/phd/summariser/clinical_sum/mimic_summarisation_clean/mimic_radiology_clean.py:20: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif selected_sec_idx != 0:


In [ ]:
# processed 88% of notes into the assessment and plan section, impression, 'progress' section (Nursing)

In [9]:
all_notes[~pd.isna(all_notes.clean_text)].shape[0] / all_notes.shape[0] * 100

87.49803103026905

In [10]:
all_notes.hadm_id.unique()

(48142,)

In [11]:
clean_notes = all_notes[~pd.isna(all_notes.clean_text)]

In [16]:
count_to_doc_counts = pd.Series(Counter([g[1].shape[0] for g in all_notes.groupby('hadm_id')])).sort_index()

In [270]:
all_notes = pd.read_csv('all_notes_partially_parsed.csv')

In [271]:
hadms_with_mult_summaries = dis_notes.hadm_id.value_counts()[dis_notes.hadm_id.value_counts() > 1].index.tolist()

In [275]:
dis_notes = dis_notes[~dis_notes.hadm_id.isin(hadms_with_mult_summaries)]

In [ ]:
dis_notes.hos_course_text_subsection

In [280]:
hadms_to_course = {r.hadm_id: r.hos_course_text_subsection for r in dis_notes.itertuples()}

In [283]:
all_notes = all_notes.sort_values(['hadm_id', 'chartdate', 'charttime'])

In [288]:
clean_notes = all_notes[~pd.isna(all_notes.clean_text)]

In [289]:
clean_notes = clean_notes.sort_values(['hadm_id', 'chartdate', 'charttime'])

In [ ]:
# maybe ignore these, although probably should probably remove the entire sentence?
# clean_notes.clean_text.str.replace('\[\*\*|\*\*\]', ' ')

In [291]:
hadms_to_dis_course = [{'hadm_id': r.hadm_id, 'text': r.hos_course_text_subsection} for r in dis_notes.itertuples() if not pd.isna(r.hos_course_text_subsection)]

In [ ]:
dis_notes.other_dis_notes_secs.iloc[0]

In [311]:
secs, sec_headers = dis_notes[dis_notes.hadm_id == 166613].iloc[0].other_dis_notes_secs

In [ ]:
clean_notes[clean_notes.hadm_id == 100003.0]

In [321]:
hadms_to_hosp_course = {}
for h in hadms_to_dis_course.keys():
    h_notes = clean_notes[clean_notes.hadm_id == h]
    secs, sec_headers = dis_notes[dis_notes.hadm_id == h].iloc[0].other_dis_notes_secs
    dis_notes_extension = '\n\n'.join([f'{h}\n{sec}' for sec, h in zip(secs, sec_headers)])
    non_dis_notes = '\n\n'.join(h_notes.apply(lambda r: f'{r.category}:{r.description}\n{r.clean_text}', axis=1))
    hadms_to_hosp_course[h] = f'{non_dis_notes}\n\nDischarge summary\n{dis_notes_extension}'

In [ ]:
# hf datasets json format
'hadm_id': for hadm_id, text in hadms_to_dis_course.items():
    

In [328]:
json.dump(hadms_to_dis_course, open('hadms_to_dis_course.json', 'w'))
json.dump(hadms_to_hosp_course, open('hadms_to_hosp_course', 'w'))